# Module 4: Hourly Updates

## Purpose
This module runs hourly to monitor and adjust prices based on:
1. **UTH (Up-Till-Hour) Performance**: Cumulative qty/retailers from start of day until current hour
2. **Last Hour Performance**: Qty/retailers for the most recent hour only

## Schedule
- Runs hourly from 12 PM to 12 AM (midnight), **except** Module 3 hours (12 PM, 3 PM, 6 PM, 9 PM)
- Also runs once at 3 AM
- Active hours: 1 PM, 2 PM, 4 PM, 5 PM, 7 PM, 8 PM, 10 PM, 11 PM, 12 AM, 3 AM

## Data Flow
```
data_extraction.ipynb → pricing_with_discount.xlsx
                              ↓
                        Module 4 (this module)
                           ├── Load p80_qty, p70_retailers, std columns
                           ├── Fetch fresh: cart rules, stocks, WAC
                           ├── Calculate new_wac (from today's PRS)
                           ├── Query UTH performance (today)
                           ├── Query Last Hour performance (today)
                           ├── Query historical hour contributions
                           ├── Calculate targets & statuses (±3 std)
                           └── Generate actions (TBD)
```

## New WAC Calculation
To avoid database lag, we calculate a fresh WAC from today's purchase receipts:
- `new_wac1` = ((stocks + reflected_qty) × wac1 + unreflected_qty × item_price) / total_qty
- `new_wac_p` = wac_p × (1 + wac1_change)
- `new_wac` = new_wac_p (with fallback to current_wac)

## Status Logic (±3 Standard Deviations)
SKU status is determined by comparing actual performance to target ± 3 std:
- **Growing**: actual > target + 3×std
- **On Track**: target - 3×std ≤ actual ≤ target + 3×std
- **Dropping**: actual < target - 3×std (minimum threshold = 1)

Standard deviation columns used:
- Qty: `std_daily_240d`
- Retailers: `std_daily_retailers_240d`

## Status Outputs
- `uth_qty_status`: growing / dropping / on_track
- `uth_rets_status`: growing / dropping / on_track
- `last_hour_qty_status`: growing / dropping / on_track
- `last_hour_rets_status`: growing / dropping / on_track


In [1]:
# =============================================================================
# IMPORTS AND SETUP
# =============================================================================
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import sys
sys.path.append('..')
import setup_environment_2
# Import queries module for Snowflake access
%run queries_module.ipynb

# Cairo timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)
CURRENT_HOUR = CAIRO_NOW.hour

print(f"Module 4: Hourly Updates")
print(f"Current Cairo Time: {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Current Hour: {CURRENT_HOUR}")


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (22.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json
Queries Module | Timezone: America/Los_Angeles
✅ UTH and Last Hour functions defined

QUERIES MODULE READY

Live Data Functions:
  • get_current_stocks()
  • get_packing_units()
  • get_current_prices()
  • get_current_wac()
  • get_current_cart_rules()

UTH Performance Functions:
  • get_uth_performance()         - UTH qty/retailers (Snowflake)
  • get_hourly_distribution()     - Historical hour contributions (Snowflake)
  • get_last_hour_performance()   - Last hour qty/retailers (DWH)

Note: Market prices use MODULE_1_INPUT data
Retailer Selection Queries defined ✓
  - get_churned_dropped_retailers()
  - get_category_not_product_retailers()
  - get_out_of_cycle_retailers()
  - get_view_no_orders_retailers()
  - get_excluded_retailers()
  - get_retailers_with_quantity_discount()
  - get_retailer_main_warehouse()
Module 4: Hourly Updates
Current Cairo Time: 2026-01-25 18:48:56
Current Hour: 18


In [2]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Input/Output files
INPUT_FILE = '../pricing_with_discount.xlsx'
OUTPUT_FILE = f'module_4_output_{CAIRO_NOW.strftime("%Y%m%d_%H%M")}.xlsx'

# Status thresholds (±3 std from target)
# - Growing: actual > target + 3*std
# - On Track: target - 3*std <= actual <= target + 3*std
# - Dropping: actual < target - 3*std (minimum threshold = 1)
STD_THRESHOLD = 3  # Number of standard deviations
MIN_DROPPING_THRESHOLD = 1  # Minimum threshold for dropping status

# Module 3 hours (skip these)
MODULE_3_HOURS = [12, 15, 18, 21]  # 12 PM, 3 PM, 6 PM, 9 PM

print(f"Input: {INPUT_FILE}")
print(f"Output: {OUTPUT_FILE}")
print(f"Status Thresholds: ±{STD_THRESHOLD} std (Dropping minimum = {MIN_DROPPING_THRESHOLD})")


Input: ../pricing_with_discount.xlsx
Output: module_4_output_20260125_1848.xlsx
Status Thresholds: ±3 std (Dropping minimum = 1)


In [5]:
df

,cohort_id,product_id,region,warehouse_id,warehouse,sku,brand,cat,wac1,wac_p,...,star_performer_flag,total_nmv_4m,no_nmv_4m,frequent_retailer_count,frequent_order_count,normal_refill,refill_stddev,current_cart_rule,commercial_min_price,active_sku_disc_pct
0,1126,10413,Upper Egypt,401,Bani sweif,مربى البوادى مشمش - 365 جم,البوادي,مربي,29.582772,26.183455,...,0,6808.00,0,5,10,6.80,4.66,12,0.00,0.310000
1,1126,972,Upper Egypt,401,Bani sweif,نواعم بسكويت- 5 ج,نواعم,بسكويت و معمول,48.330000,46.880100,...,0,29434.25,0,9,29,4.41,3.89,695,48.33,0.360000
2,703,11726,Delta West,337,El-Mahala,مناديل زينة تريو تواليت عرض 5 + 1 رول هدية 6 بكرة,زينه,ورقيات,308.210129,280.278564,...,0,34846.00,0,13,31,2.06,1.46,25,0.00,1.080000
3,703,11726,Delta West,8,Tanta,مناديل زينة تريو تواليت عرض 5 + 1 رول هدية 6 بكرة,زينه,ورقيات,308.210129,280.278564,...,0,66585.25,0,26,81,1.75,1.17,25,0.00,1.080000
4,700,2104,Cairo,1,Mostorod,البوادى حلاوة سادة- 550 جم,البوادي,حلاوة طحينية,90.730000,82.974183,...,0,314845.50,0,64,245,4.44,3.35,12,0.00,1.290388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28377,702,11116,Alexandria,797,Khorshed Alex,اندومى شعرية مقلية عرض 44 كيس - 80 جم,اندومي,شعرية سريعة التحضير,377.000000,357.893835,...,0,52583.00,0,17,51,1.25,0.72,25,0.00,1.580000
28378,703,11092,Delta West,337,El-Mahala,اندومى فراخ بالكارى عرض 44 كيس - 75 جم,اندومي,شعرية سريعة التحضير,377.000000,354.379619,...,0,43680.50,0,16,35,1.49,0.89,25,0.00,0.000000
28379,703,11092,Delta West,8,Tanta,اندومى فراخ بالكارى عرض 44 كيس - 75 جم,اندومي,شعرية سريعة التحضير,377.000000,354.379619,...,0,22635.50,0,4,8,1.25,0.46,25,0.00,0.000000
28380,704,34,Delta East,339,Mansoura FC,مكرونة حواء مرمرية - 400 جم,حواء,مكرونة,247.029014,247.029014,...,0,137161.25,0,17,66,1.45,0.86,129,0.00,0.390000


In [6]:
# =============================================================================
# LOAD DATA FROM DATA EXTRACTION
# =============================================================================
print("Loading data from data_extraction output...")
df = pd.read_excel(INPUT_FILE)
print(f"Loaded {len(df)} records")

# Ensure required columns exist with proper types
df['p80_daily_240d'] = pd.to_numeric(df.get('p80_daily_240d', 0), errors='coerce').fillna(0)
df['p70_daily_retailers_240d'] = pd.to_numeric(df.get('p70_daily_retailers_240d', 1), errors='coerce').fillna(1)
df['std_daily_240d'] = pd.to_numeric(df.get('std_daily_240d', 0), errors='coerce').fillna(0)
df['std_daily_retailers_240d'] = pd.to_numeric(df.get('std_daily_retailers_240d', 0), errors='coerce').fillna(0)
df['warehouse_id'] = df['warehouse_id'].astype(int)
df['product_id'] = df['product_id'].astype(int)
df['cohort_id'] = df['cohort_id'].astype(int) if 'cohort_id' in df.columns else None

# Get category for hourly distribution merge
if 'cat' not in df.columns and 'category' in df.columns:
    df['cat'] = df['category']

print(f"\nP80 Qty Stats: min={df['p80_daily_240d'].min():.1f}, max={df['p80_daily_240d'].max():.1f}, mean={df['p80_daily_240d'].mean():.1f}")
print(f"P70 Retailers Stats: min={df['p70_daily_retailers_240d'].min():.1f}, max={df['p70_daily_retailers_240d'].max():.1f}, mean={df['p70_daily_retailers_240d'].mean():.1f}")
print(f"Std Qty Stats: min={df['std_daily_240d'].min():.1f}, max={df['std_daily_240d'].max():.1f}, mean={df['std_daily_240d'].mean():.1f}")
print(f"Std Retailers Stats: min={df['std_daily_retailers_240d'].min():.1f}, max={df['std_daily_retailers_240d'].max():.1f}, mean={df['std_daily_retailers_240d'].mean():.1f}")

# =============================================================================
# GET FRESH DATA FROM QUERIES MODULE (Snowflake)
# =============================================================================

# 1. Current Cart Rules
df_cart_rules = get_current_cart_rules()

# Merge with main df (by cohort_id + product_id)
if 'cohort_id' in df.columns and len(df_cart_rules) > 0:
    df = df.drop(columns=['current_cart_rule'], errors='ignore')
    df = df.merge(df_cart_rules, on=['cohort_id', 'product_id'], how='left')
    df['current_cart_rule'] = df['current_cart_rule'].fillna(999)
    print(f"✅ Merged cart rules: {len(df)} records")
else:
    df['current_cart_rule'] = df.get('current_cart_rule', 999)

# 2. Current Stocks
df_stocks = get_current_stocks()

# Merge stocks (by warehouse_id + product_id)
if len(df_stocks) > 0:
    df = df.drop(columns=['stocks'], errors='ignore')
    df = df.merge(df_stocks, on=['warehouse_id', 'product_id'], how='left')
    df['stocks'] = df['stocks'].fillna(0)
    print(f"✅ Merged stocks: {len(df)} records")
else:
    df['stocks'] = df.get('stocks', 0)

# 3. Current WAC (Weighted Average Cost)
df_wac = get_current_wac()

# Merge WAC (by warehouse_id + product_id)
if len(df_wac) > 0:
    df = df.drop(columns=['wac_p'], errors='ignore')
    df = df.merge(df_wac, on=['product_id'], how='left')
    df['wac_p'] = df['wac_p'].fillna(0)
    print(f"✅ Merged WAC: {len(df)} records")
else:
    df['wac_p'] = df.get('wac_p', 0)

# 4. Current Prices
df_prices = get_current_prices()

# Merge prices (by cohort_id + product_id)
if len(df_prices) > 0:
    df = df.drop(columns=['current_price'], errors='ignore')
    df = df.merge(df_prices[['cohort_id', 'product_id', 'current_price']], on=['cohort_id', 'product_id'], how='left')
    df['current_price'] = df['current_price'].fillna(0)
    print(f"✅ Merged current prices: {len(df)} records")
else:
    df['current_price'] = df.get('current_price', 0)

print(f"\nCurrent Stock Stats: min={df['stocks'].min():.0f}, max={df['stocks'].max():.0f}, mean={df['stocks'].mean():.1f}")
print(f"Current WAC Stats: min={df['wac_p'].min():.2f}, max={df['wac_p'].max():.2f}, mean={df['wac_p'].mean():.2f}")
print(f"Current Price Stats: min={df['current_price'].min():.2f}, max={df['current_price'].max():.2f}, mean={df['current_price'].mean():.2f}")


Loading data from data_extraction output...
Loaded 28437 records

P80 Qty Stats: min=0.0, max=1652.8, mean=9.9
P70 Retailers Stats: min=0.0, max=146.6, mean=2.6
Std Qty Stats: min=0.0, max=2574.8, mean=6.2
Std Retailers Stats: min=0.0, max=76.2, mean=1.4
Fetching current cart rules...
  Loaded 72968 records
✅ Merged cart rules: 28437 records
Fetching current stocks...
  Loaded 1848028 records
✅ Merged stocks: 28437 records
Fetching current WAC...
  Loaded 8147 records
✅ Merged WAC: 28437 records
Fetching current prices...
  Loaded 116458 records
✅ Merged current prices: 28437 records

Current Stock Stats: min=0, max=28369, mean=73.0
Current WAC Stats: min=0.01, max=2082.02, mean=227.64
Current Price Stats: min=0.00, max=2203.25, mean=241.78


In [7]:
# =============================================================================
# CALCULATE NEW WAC (Accounting for Today's Unreflected Purchases)
# =============================================================================
# This calculates a fresh WAC that accounts for today's purchase receipts
# that haven't been reflected in the database yet (to avoid lag)

print("Calculating new WAC from today's purchase receipts...")

# 1. Query today's PRS data (purchases by product_id)
prs_query = '''
WITH prs AS (
    SELECT DISTINCT 
        product_purchased_receipts.purchased_receipt_id,
        products.id AS product_id,
        product_purchased_receipts.basic_unit_count,
        product_purchased_receipts.purchased_item_count * product_purchased_receipts.basic_unit_count AS purchase_min_count,
        product_purchased_receipts.final_price / product_purchased_receipts.purchased_item_count AS final_item_price
    FROM product_purchased_receipts
    LEFT JOIN products ON products.id = product_purchased_receipts.product_id
    LEFT JOIN purchased_receipts ON purchased_receipts.id = product_purchased_receipts.purchased_receipt_id
    WHERE product_purchased_receipts.purchased_item_count <> 0
        AND purchased_receipts.purchased_receipt_status_id IN (4,5,7)
        AND purchased_receipts.date::date >= current_date
        AND product_purchased_receipts.final_price > 0 
)
SELECT 
    product_id,
    SUM(purchase_min_count) AS all_day_qty,
    AVG(final_item_price / basic_unit_count) AS item_price
FROM prs
GROUP BY 1
'''
df_prs = setup_environment_2.dwh_pg_query(prs_query, columns=['product_id', 'all_day_qty', 'item_price'])
df_prs['product_id'] = pd.to_numeric(df_prs['product_id'])
df_prs['all_day_qty'] = pd.to_numeric(df_prs['all_day_qty'])
df_prs['item_price'] = pd.to_numeric(df_prs['item_price'])
print(f"  Loaded {len(df_prs)} PRS records (today's purchases)")

# 2. Query what's already reflected in WAC tracker
reflected_query = '''
SELECT
    t_product_id AS product_id,
    s_beg_stock AS av_stocks,
    p_purchased_item_count AS pr_qty
FROM finance.wac_tracker wt
WHERE wt.t_date::date = CURRENT_DATE
    AND p_purchased_item_count > 0 
'''
try:
    df_reflected = setup_environment_2.dwh_pg_query(reflected_query, columns=['product_id', 'av_stocks', 'pr_qty'])
    df_reflected['product_id'] = pd.to_numeric(df_reflected['product_id'])
    df_reflected['av_stocks'] = pd.to_numeric(df_reflected['av_stocks'])
    df_reflected['pr_qty'] = pd.to_numeric(df_reflected['pr_qty'])
    print(f"  Loaded {len(df_reflected)} WAC tracker records (already reflected)")
except:
    df_reflected = pd.DataFrame(columns=['product_id', 'av_stocks', 'pr_qty'])
    print("  No WAC tracker records found for today")

# 3. Query base WAC (wac1, wac_p) from all_cogs
wac_base_query = '''
SELECT 
    f.product_id,
    f.wac1,
    f.wac_p
FROM finance.all_cogs f
JOIN products ON products.id = f.product_id
JOIN categories ON products.category_id = categories.id
WHERE current_timestamp BETWEEN f.from_date AND f.to_date 
    AND NOT categories.name_ar IN (
        SELECT categories.name_ar AS cat
        FROM categories
        JOIN sections s ON s.id = categories.section_id
        WHERE categories.name_ar LIKE '%سايب%'
            OR categories.name_ar LIKE '%بالتة%'
            OR categories.section_id IN (225, 318, 285, 121, 87, 351, 417)
    )
'''
df_wac_base = query_snowflake(wac_base_query)
df_wac_base['product_id'] = pd.to_numeric(df_wac_base['product_id'])
df_wac_base['wac1'] = pd.to_numeric(df_wac_base['wac1'])
df_wac_base['wac_p'] = pd.to_numeric(df_wac_base['wac_p'])
print(f"  Loaded {len(df_wac_base)} base WAC records from all_cogs")

# 4. Merge and calculate new WAC
df_wac_calc = df_wac_base.merge(df_prs, on='product_id', how='left')
df_wac_calc = df_wac_calc.merge(df_reflected, on='product_id', how='left')
df_wac_calc = df_wac_calc.fillna(0)

# Use current_stock from main df if av_stocks is 0
df_stock_lookup = df[['product_id', 'stocks']].drop_duplicates()
df_stock_lookup = df_stock_lookup.groupby(['product_id'])['stocks'].sum().reset_index()
df_wac_calc = df_wac_calc.merge(df_stock_lookup, on='product_id', how='left')
df_wac_calc['av_stocks'] = df_wac_calc.apply(
    lambda row: row['stocks'] if row['av_stocks'] == 0 else row['av_stocks'], axis=1)

# Calculate not reflected qty (purchases not yet in WAC tracker)
df_wac_calc['not_reflected_qty'] = df_wac_calc['all_day_qty'] - df_wac_calc['pr_qty']
df_wac_calc['not_reflected_qty'] = df_wac_calc['not_reflected_qty'].clip(lower=0)  # Can't be negative

# Calculate new WAC
# new_wac1 = ((stocks + pr_qty) * wac1 + not_reflected_qty * item_price) / (stocks + pr_qty + not_reflected_qty)
denominator = df_wac_calc['av_stocks'] + df_wac_calc['pr_qty'] + df_wac_calc['not_reflected_qty']
numerator = ((df_wac_calc['av_stocks'] + df_wac_calc['pr_qty']) * df_wac_calc['wac1'] + 
             df_wac_calc['not_reflected_qty'] * df_wac_calc['item_price'])

df_wac_calc['new_wac1'] = np.where(denominator > 0, numerator / denominator, df_wac_calc['wac1'])

# Calculate wac1 change and new_wac_p
df_wac_calc['wac1_change'] = np.where(
    df_wac_calc['wac1'] > 0,
    (df_wac_calc['new_wac1'] - df_wac_calc['wac1']) / df_wac_calc['wac1'],
    0
)
df_wac_calc['new_wac_p'] = df_wac_calc['wac_p'] * (1 + df_wac_calc['wac1_change'])
df_wac_calc['new_wac_p'] = df_wac_calc['new_wac_p'].fillna(df_wac_calc['wac_p'])

# Prepare final new_wac dataframe
df_new_wac = df_wac_calc[['product_id','new_wac_p', 'not_reflected_qty']].copy()
df_new_wac=df_new_wac.drop_duplicates()

# 5. Merge new_wac into main dataframe
df = df.drop(columns=['new_wac_p','not_reflected_qty'], errors='ignore')
df = df.merge(df_new_wac, on='product_id', how='left')
df['new_wac'] = df['new_wac_p'].fillna(df['wac_p'])  # Fallback to current_wac if no new_wac

print(f"\n✅ New WAC calculated: {len(df)} records")
print(f"   Products with unreflected purchases: {(df['not_reflected_qty'] > 0).sum()}")
print(f"   New WAC Stats: min={df['new_wac'].min():.2f}, max={df['new_wac'].max():.2f}, mean={df['new_wac'].mean():.2f}")


Calculating new WAC from today's purchase receipts...
  Loaded 1062 PRS records (today's purchases)
  Loaded 647 WAC tracker records (already reflected)
  Loaded 7688 base WAC records from all_cogs

✅ New WAC calculated: 28437 records
   Products with unreflected purchases: 8075
   New WAC Stats: min=0.01, max=2082.02, mean=227.68


In [9]:
# =============================================================================
# QUERY 1: TODAY'S UTH (Up-Till-Hour) PERFORMANCE
# =============================================================================
# Gets cumulative qty and retailers from start of day until current hour
# Uses get_uth_performance() from queries_module

df_uth_today = get_uth_performance()


Fetching UTH performance from Snowflake...
  Loaded 7683 UTH records


In [10]:
# =============================================================================
# QUERY 2: TODAY'S LAST HOUR PERFORMANCE (from DWH)
# =============================================================================
# Gets qty and retailers for the PREVIOUS hour only (not cumulative)
# Uses get_last_hour_performance() from queries_module (DWH/PostgreSQL)

df_last_hour = get_last_hour_performance()


Fetching last hour performance from DWH...
  Loaded 1803 last hour records from DWH


In [11]:
# =============================================================================
# QUERY 3: HISTORICAL HOURLY DISTRIBUTION (Last 4 Months) - By Category & Warehouse
# =============================================================================
# Gets:
# - avg_uth_pct_qty/retailers: Average contribution of hours 0 to (current_hour-1) to daily total
# - avg_last_hour_pct_qty/retailers: Average contribution of (current_hour-1) alone to daily total
# Uses get_hourly_distribution() from queries_module

df_hourly_dist = get_hourly_distribution()
print(f"\nAvg UTH % (qty): {df_hourly_dist['avg_uth_pct_qty'].mean()*100:.1f}%")
print(f"Avg Last Hour % (qty): {df_hourly_dist['avg_last_hour_pct_qty'].mean()*100:.1f}%")


Fetching hourly distribution from Snowflake...
  Loaded 770 hourly distribution records

Avg UTH % (qty): 61.8%
Avg Last Hour % (qty): 6.5%


In [12]:
# =============================================================================
# MERGE DATA
# =============================================================================
print("Merging performance data with base data...")

# Merge UTH today data
if len(df_uth_today) > 0:
    df = df.merge(df_uth_today, on=['warehouse_id', 'product_id'], how='left')
else:
    df['uth_qty'] = 0
    df['uth_nmv'] = 0
    df['uth_retailers'] = 0

# Merge last hour data
if len(df_last_hour) > 0:
    df = df.merge(df_last_hour, on=['warehouse_id', 'product_id'], how='left')
else:
    df['last_hour_qty'] = 0
    df['last_hour_nmv'] = 0
    df['last_hour_retailers'] = 0

# Merge hourly distribution (by warehouse_id + cat)
if len(df_hourly_dist) > 0:
    df = df.merge(df_hourly_dist, on=['warehouse_id', 'cat'], how='left')
else:
    df['avg_uth_pct_qty'] = 0.5
    df['avg_uth_pct_retailers'] = 0.5
    df['avg_last_hour_pct_qty'] = 0.05
    df['avg_last_hour_pct_retailers'] = 0.05

# Fill NaN values
df['uth_qty'] = df['uth_qty'].fillna(0)
df['uth_nmv'] = df['uth_nmv'].fillna(0)
df['uth_retailers'] = df['uth_retailers'].fillna(0)
df['last_hour_qty'] = df['last_hour_qty'].fillna(0)
df['last_hour_nmv'] = df['last_hour_nmv'].fillna(0)
df['last_hour_retailers'] = df['last_hour_retailers'].fillna(0)
df['avg_uth_pct_qty'] = df['avg_uth_pct_qty'].fillna(0.5)
df['avg_uth_pct_retailers'] = df['avg_uth_pct_retailers'].fillna(0.5)
df['avg_last_hour_pct_qty'] = df['avg_last_hour_pct_qty'].fillna(0.05)
df['avg_last_hour_pct_retailers'] = df['avg_last_hour_pct_retailers'].fillna(0.05)

print(f"✅ Merged data: {len(df)} records")
print(f"\nUTH Qty Stats: min={df['uth_qty'].min():.0f}, max={df['uth_qty'].max():.0f}, mean={df['uth_qty'].mean():.1f}")
print(f"Last Hour Qty Stats: min={df['last_hour_qty'].min():.0f}, max={df['last_hour_qty'].max():.0f}, mean={df['last_hour_qty'].mean():.1f}")
print(f"Current Cart Rule Stats: min={df['current_cart_rule'].min():.0f}, max={df['current_cart_rule'].max():.0f}, mean={df['current_cart_rule'].mean():.1f}")


Merging performance data with base data...
✅ Merged data: 28437 records

UTH Qty Stats: min=0, max=347, mean=1.8
Last Hour Qty Stats: min=0, max=112, mean=0.2
Current Cart Rule Stats: min=1, max=10000, mean=88.0


In [13]:
# =============================================================================
# CALCULATE TARGETS AND STATUSES
# =============================================================================
print("Calculating UTH and Last Hour targets and statuses...")

def get_status_std(actual, target, std, is_qty=True):
    """
    Determine status based on ±3 std from target.
    - Growing: actual > target + 3*std
    - On Track: target - 3*std <= actual <= target + 3*std
    - Dropping: actual < target - 3*std (minimum threshold = 1)
    
    Parameters:
    - actual: actual performance value
    - target: target value (p80 for qty, p70 for retailers)
    - std: standard deviation (std_daily_240d or std_daily_retailers_240d)
    - is_qty: True for qty metrics, False for retailer metrics
    """
    upper_bound = target + STD_THRESHOLD * std
    lower_bound = max(target - STD_THRESHOLD * std, MIN_DROPPING_THRESHOLD)
    
    if actual > upper_bound:
        return 'growing'
    elif actual < lower_bound:
        return 'dropping'
    else:
        return 'on_track'

# Calculate UTH targets
# UTH target = p80_qty * avg_uth_pct (historical % of day that should be done by now)
df['uth_qty_target'] = df['p80_daily_240d'] * df['avg_uth_pct_qty']
df['uth_rets_target'] = df['p70_daily_retailers_240d'] * df['avg_uth_pct_retailers']

# Calculate UTH std thresholds (scaled by UTH percentage)
df['uth_qty_std'] = df['std_daily_240d'] * df['avg_uth_pct_qty']
df['uth_rets_std'] = df['std_daily_retailers_240d'] * df['avg_uth_pct_retailers']

# Calculate Last Hour targets
# Last hour target = p80_qty * avg_last_hour_pct (historical % of day for this hour)
df['last_hour_qty_target'] = df['p80_daily_240d'] * df['avg_last_hour_pct_qty']
df['last_hour_rets_target'] = df['p70_daily_retailers_240d'] * df['avg_last_hour_pct_retailers']

# Calculate Last Hour std thresholds (scaled by last hour percentage)
df['last_hour_qty_std'] = df['std_daily_240d'] * df['avg_last_hour_pct_qty']
df['last_hour_rets_std'] = df['std_daily_retailers_240d'] * df['avg_last_hour_pct_retailers']

# Calculate statuses using ±3 std thresholds
df['uth_qty_status'] = df.apply(
    lambda row: get_status_std(row['uth_qty'], row['uth_qty_target'], row['uth_qty_std'], is_qty=True), axis=1)
df['uth_rets_status'] = df.apply(
    lambda row: get_status_std(row['uth_retailers'], row['uth_rets_target'], row['uth_rets_std'], is_qty=False), axis=1)
df['last_hour_qty_status'] = df.apply(
    lambda row: get_status_std(row['last_hour_qty'], row['last_hour_qty_target'], row['last_hour_qty_std'], is_qty=True), axis=1)
df['last_hour_rets_status'] = df.apply(
    lambda row: get_status_std(row['last_hour_retailers'], row['last_hour_rets_target'], row['last_hour_rets_std'], is_qty=False), axis=1)

print(f"✅ Targets and statuses calculated using ±{STD_THRESHOLD} std thresholds")

# Summary
print(f"\n{'='*60}")
print("UTH STATUS DISTRIBUTION")
print(f"{'='*60}")
print(f"\nUTH Qty Status:")
print(df['uth_qty_status'].value_counts().to_string())
print(f"\nUTH Retailers Status:")
print(df['uth_rets_status'].value_counts().to_string())

print(f"\n{'='*60}")
print("LAST HOUR STATUS DISTRIBUTION")
print(f"{'='*60}")
print(f"\nLast Hour Qty Status:")
print(df['last_hour_qty_status'].value_counts().to_string())
print(f"\nLast Hour Retailers Status:")
print(df['last_hour_rets_status'].value_counts().to_string())


Calculating UTH and Last Hour targets and statuses...
✅ Targets and statuses calculated using ±3 std thresholds

UTH STATUS DISTRIBUTION

UTH Qty Status:
uth_qty_status
dropping    21186
on_track     6546
growing       705

UTH Retailers Status:
uth_rets_status
dropping    21187
on_track     6340
growing       910

LAST HOUR STATUS DISTRIBUTION

Last Hour Qty Status:
last_hour_qty_status
dropping    26746
on_track     1005
growing       686

Last Hour Retailers Status:
last_hour_rets_status
dropping    26746
growing       920
on_track      771


In [14]:
# =============================================================================
# FETCH MARKET DATA AND MARGIN TIERS
# =============================================================================
print("Fetching market data and margin tiers...")

# Import market_data_module for get_market_data()
%run market_data_module.ipynb

# 1. Get margin tiers (already available from queries_module)
df_margin_tiers = get_margin_tiers()
print(f"✅ Loaded {len(df_margin_tiers)} margin tier records")

# 2. Get market data
df_market = get_market_data()
print(f"✅ Loaded {len(df_market)} market data records")

# Verify cohort_id is available in all dataframes
print(f"\nChecking cohort_id availability:")
print(f"  - df has cohort_id: {'cohort_id' in df.columns}")
print(f"  - df_margin_tiers has cohort_id: {'cohort_id' in df_margin_tiers.columns}")
print(f"  - df_market has cohort_id: {'cohort_id' in df_market.columns}")

# 3. Merge margin tiers with df (by cohort_id + product_id) - ALL COLUMNS
merge_keys = ['cohort_id', 'product_id']
df = df.drop(columns=[c for c in df_margin_tiers.columns if c in df.columns and c not in merge_keys], errors='ignore')
df = df.merge(df_margin_tiers, on=merge_keys, how='left')
print(f"\n✅ Merged margin tiers: {len(df)} records")
print(f"   Margin tier columns added: {[c for c in df_margin_tiers.columns if c not in merge_keys]}")

# 4. Merge market data with df (by cohort_id + product_id) - ALL COLUMNS
df = df.drop(columns=[c for c in df_market.columns if c in df.columns and c not in merge_keys], errors='ignore')
df = df.merge(df_market, on=merge_keys, how='left')
print(f"✅ Merged market data: {len(df)} records")
print(f"   Market columns added: {[c for c in df_market.columns if c not in merge_keys]}")

# 5. Calculate current margin based on current_price and new_wac
df['current_margin'] = (df['current_price'] - df['new_wac']) / df['current_price']
df['current_margin'] = df['current_margin'].fillna(0)

print(f"\nMargin Stats: min={df['current_margin'].min():.2%}, max={df['current_margin'].max():.2%}, mean={df['current_margin'].mean():.2%}")

# =============================================================================
# FILTER SKUS FOR ACTION
# =============================================================================
# Filter SKUs that need action based on:
# Condition A: new_wac > wac_p * 1.005 (WAC increased by more than 0.5%)
# Condition B: Last hour growing (qty OR rets) AND UTH on_track/growing (qty AND rets)

print("Filtering SKUs for action...")

# Condition A: WAC increased significantly
condition_a = df['new_wac'] > df['wac_p'] * 1.005

# Condition B: Last hour growing AND UTH performing well
last_hour_growing = (df['last_hour_qty_status'] == 'growing') | (df['last_hour_rets_status'] == 'growing')
uth_good = (df['uth_qty_status'].isin(['on_track', 'growing'])) & (df['uth_rets_status'].isin(['on_track', 'growing']))
condition_b = last_hour_growing & uth_good

# Combine conditions (A OR B)
df['needs_action'] = condition_a | condition_b
df['action_reason'] = 'none'
df.loc[condition_a & ~condition_b, 'action_reason'] = 'wac_increase'
df.loc[~condition_a & condition_b, 'action_reason'] = 'growing_performance'
df.loc[condition_a & condition_b, 'action_reason'] = 'both'

# Define important columns for df_action
action_columns = [
    # Identification
    'warehouse_id', 'product_id', 'cohort_id', 'sku', 'region', 'brand', 'cat',
    # Price & Cost
    'current_price', 'wac_p', 'new_wac', 'current_margin',
    # Inventory & Rules
    'stocks', 'current_cart_rule',
    # Performance Targets
    'p80_daily_240d', 'p70_daily_retailers_240d', 'std_daily_240d', 'std_daily_retailers_240d',
    # Refill columns for cart rule calculation
    'normal_refill', 'refill_stddev', 'target_margin',
    # UTH Status
    'uth_qty', 'uth_qty_status', 'uth_retailers', 'uth_rets_status',
    # Last Hour Status
    'last_hour_qty', 'last_hour_qty_status', 'last_hour_retailers', 'last_hour_rets_status',
    # Market Margins
    'below_market', 'market_min', 'market_25', 'market_50', 'market_75', 'market_max', 'above_market',
    # Margin Tiers (excluding: effective_min_margin, max_boundary, margin_step, margin_tier_below)
    'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5',
    'margin_tier_above_1', 'margin_tier_above_2',
    # Action Flags
    'needs_action', 'action_reason'
]

# Filter to only columns that exist in df
action_columns = [c for c in action_columns if c in df.columns]

# Filter to action SKUs with selected columns only
df_action = df[df['needs_action']][action_columns].copy()

print(f"\n{'='*60}")
print("SKUs FILTERED FOR ACTION")
print(f"{'='*60}")
print(f"\nTotal SKUs needing action: {len(df_action)} out of {len(df)} ({len(df_action)/len(df)*100:.1f}%)")
print(f"\nBreakdown by reason:")
print(df_action['action_reason'].value_counts().to_string())

print(f"\n--- Condition A (WAC increase > 0.5%): {condition_a.sum()} SKUs")
print(f"--- Condition B (Last hour growing + UTH good): {condition_b.sum()} SKUs")

# Show sample
print(f"\n{'='*60}")
print("SAMPLE ACTION SKUs")
print(f"{'='*60}")
print(f"\nTotal columns in df_action: {len(df_action.columns)}")
print(f"Columns: {df_action.columns.tolist()}")
display(df_action.head(10))


Fetching market data and margin tiers...
/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json
Market Data Module loaded at 2026-01-25 18:57:29 Cairo time
Snowflake timezone: America/Los_Angeles
All queries defined ✓
Helper functions defined ✓
get_market_data() function defined ✓
get_margin_tiers() function defined ✓

MARKET DATA MODULE READY

Available functions (NO INPUT REQUIRED):
  - get_market_data()   : Fetch and process all market prices
  - get_margin_tiers()  : Fetch and calculate margin tiers

Usage:
  %run market_data_module.ipynb
  df_market = get_market_data()
  df_tiers = get_margin_tiers()

FETCHING MARGIN TIERS
Timestamp: 2026-01-25 18:57:30 Cairo time

Step 1: Fetching margin boundaries from PRODUCT_STATISTICS...
    Loaded 18201 records

Step 2: Adding cohort IDs...
    Records with cohorts: 25059

Step 3: Calculating margin tiers...

MARGIN TIERS COMPLETE
Total records: 25059

Margin Tier Structure:
  margin_tier_below:   effective_min - step (1 below)
  m

/tmp/ipykernel_19587/1916930114.py:138: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


    Records after group processing: 25679

Step 5: Adding WAC and margin data...
    Records with WAC: 25463

Step 6: Filtering by price coverage...
    Records after price coverage filter: 13656

Step 7: Calculating price percentiles...
    Records after price analysis: 12917

Step 8: Converting prices to margins...

MARKET DATA COMPLETE
Total records: 12917
  - With marketplace prices: 12344
  - With scrapped prices: 6477
  - With Ben Soliman prices: 8756
✅ Loaded 12917 market data records

Checking cohort_id availability:
  - df has cohort_id: True
  - df_margin_tiers has cohort_id: True
  - df_market has cohort_id: True

✅ Merged margin tiers: 28437 records
   Margin tier columns added: ['region', 'optimal_bm', 'min_boundary', 'max_boundary', 'median_bm', 'effective_min_margin', 'margin_step', 'margin_tier_below', 'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2']
✅ Merged market data: 28437 records
   

,warehouse_id,product_id,cohort_id,sku,region,brand,cat,current_price,wac_p,new_wac,...,above_market,margin_tier_1,margin_tier_2,margin_tier_3,margin_tier_4,margin_tier_5,margin_tier_above_1,margin_tier_above_2,needs_action,action_reason
19,8,7705,703,مكرونة بساطة فرن - 350 جم,Delta West,بساطة,مكرونة,166.75,153.926030,153.904189,...,0.076905,0.046305,0.055545,0.064786,0.074027,0.083267,0.092508,0.101748,True,growing_performance
23,962,3976,701,رويال أعشاب نعناع - 20 فتلة,NaN,رويال,اعشاب,18.25,16.666788,16.666791,...,NaN,0.066853,0.076871,0.086888,0.096905,0.106923,0.116940,0.126957,True,growing_performance
50,703,12595,1123,أد - مى باربيكيو 430 جم,NaN,اد-مي,صلصة و صوص,584.50,465.947369,475.231675,...,NaN,0.025211,0.035783,0.046355,0.056928,0.067500,0.078072,0.088645,True,wac_increase
52,1,11770,700,اوكسى يدوى نسيم الشرق - 250 جم,NaN,اوكسي,منظفات,138.25,130.477940,138.251835,...,NaN,0.050022,0.060016,0.070011,0.080005,0.090000,0.099995,0.109989,True,wac_increase
69,797,10083,702,شوكولاتة غامقة دريم خام للطبخ - 200 جم,Alexandria,دريم,حلويات سريعة التحضير,50.25,47.496006,48.249869,...,0.144216,0.056625,0.065077,0.073529,0.081982,0.090434,0.098887,0.107339,True,wac_increase
75,962,11886,701,صابون لوكس سحر الاوركيد - 165 جم,NaN,لوكس,صابون,152.25,146.622245,146.622245,...,NaN,0.028000,0.034750,0.041500,0.048250,0.055000,0.061750,0.068500,True,growing_performance
83,1,12656,700,-أريال يدوى لافندر 1 كجم,Cairo,اريال,منظفات,375.75,334.774386,334.774386,...,0.257844,0.040989,0.051367,0.061745,0.072122,0.082500,0.092878,0.103255,True,growing_performance
124,337,12597,703,اد مى مايونيز بالثوم سكويز - 360 جم,NaN,اد-مي,صلصة و صوص,601.25,557.233669,567.256078,...,NaN,0.031500,0.040066,0.048631,0.057197,0.065763,0.074329,0.082894,True,wac_increase
125,8,12597,703,اد مى مايونيز بالثوم سكويز - 360 جم,NaN,اد-مي,صلصة و صوص,601.25,557.233669,567.256078,...,NaN,0.031500,0.040066,0.048631,0.057197,0.065763,0.074329,0.082894,True,wac_increase
156,1,11350,700,الضحى خلطة توابل الحواوشى - 30 جم,NaN,الضحى,مرقة وخلطات,134.00,131.099992,131.099992,...,NaN,0.019552,0.024986,0.030420,0.035854,0.041288,0.046722,0.052156,True,growing_performance


In [ ]:
# =============================================================================
# ACTION LOGIC
# =============================================================================
print("Calculating actions for filtered SKUs...")

# Step 0: Preparation - Calculate normal_refill_3std
df_action['normal_refill_3std'] = df_action['normal_refill'] + 3 * df_action['refill_stddev']

# Initialize output columns
df_action['new_price'] = np.nan
df_action['new_cart_rule'] = np.nan
df_action['price_action'] = 'none'
df_action['cart_rule_action'] = 'none'

# =============================================================================
# HELPER: Calculate all price tiers from margins using wac_p
# =============================================================================
def calculate_price_from_margin(wac, margin):
    """Calculate price from WAC and margin: price = wac / (1 - margin)"""
    if pd.isna(margin) or margin >= 1:
        return np.nan
    return wac / (1 - margin)

# Calculate market prices (using wac_p)
market_margin_cols = ['market_min', 'market_25', 'market_50', 'market_75', 'market_max']
for col in market_margin_cols:
    price_col = f'{col}_price'
    df_action[price_col] = df_action.apply(
        lambda row: calculate_price_from_margin(row['wac_p'], row[col]), axis=1)

# Calculate margin tier prices (using wac_p)
tier_margin_cols = ['margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4', 'margin_tier_5',
                    'margin_tier_above_1', 'margin_tier_above_2']
for col in tier_margin_cols:
    price_col = f'{col}_price'
    df_action[price_col] = df_action.apply(
        lambda row: calculate_price_from_margin(row['wac_p'], row[col]), axis=1)

# =============================================================================
# HELPER: Find first price above threshold
# =============================================================================
def find_first_price_above(row, threshold, price_cols):
    """Find the first price in price_cols that is above threshold."""
    for col in price_cols:
        price = row.get(col, np.nan)
        if pd.notna(price) and price > threshold:
            return price
    return np.nan

# Define price column order (market first, then tiers)
market_price_cols = ['market_min_price', 'market_25_price', 'market_50_price', 'market_75_price', 'market_max_price']
tier_price_cols = ['margin_tier_1_price', 'margin_tier_2_price', 'margin_tier_3_price', 
                   'margin_tier_4_price', 'margin_tier_5_price', 
                   'margin_tier_above_1_price', 'margin_tier_above_2_price']
all_price_cols = market_price_cols + tier_price_cols

# =============================================================================
# STEP 1: WAC Increase Actions (action_reason = 'both' or 'wac_increase')
# =============================================================================
# Only apply if: current_price < new_wac AND current_price > wac_p
# (price is squeezed between old and new WAC)
wac_increase_mask = (
    df_action['action_reason'].isin(['both', 'wac_increase']) &
    (df_action['current_price'] < df_action['new_wac']) &
    (df_action['current_price'] > df_action['wac_p'])
)

def get_wac_increase_price(row):
    """Find first market/tier price above current_price."""
    # Find first price above current_price (not new_wac)
    new_price = find_first_price_above(row, row['current_price'], all_price_cols)
    return new_price

df_action.loc[wac_increase_mask, 'new_price'] = df_action[wac_increase_mask].apply(get_wac_increase_price, axis=1)
df_action.loc[wac_increase_mask, 'price_action'] = 'wac_increase'

print(f"  WAC increase actions: {wac_increase_mask.sum()} SKUs")
print(f"    (SKUs where wac_p < current_price < new_wac)")

# =============================================================================
# STEP 2: Growing Performance Actions (action_reason = 'growing_performance')
# =============================================================================
growing_mask = df_action['action_reason'] == 'growing_performance'

# Case A: Retailers growing (with or without qty)
rets_growing_mask = growing_mask & (df_action['last_hour_rets_status'] == 'growing')

def get_rets_growing_price(row):
    """Find price for rets growing: market/tier >= current_price * 1.005, or fallback to target margin."""
    min_price = row['current_price'] * 1.005
    
    # Try market prices first
    new_price = find_first_price_above(row, min_price, market_price_cols)
    if pd.notna(new_price):
        return new_price
    
    # Try tier prices
    new_price = find_first_price_above(row, min_price, tier_price_cols)
    if pd.notna(new_price):
        return new_price
    
    # Fallback: wac_p / (1 - (target_margin * 1.15))
    target_margin = row.get('target_margin', 0)
    if pd.notna(target_margin) and target_margin * 1.15 < 1:
        return row['wac_p'] / (1 - (target_margin * 1.15))
    
    return np.nan

df_action.loc[rets_growing_mask, 'new_price'] = df_action[rets_growing_mask].apply(get_rets_growing_price, axis=1)
df_action.loc[rets_growing_mask, 'price_action'] = 'rets_growing'

print(f"  Rets growing actions (price increase): {rets_growing_mask.sum()} SKUs")

# Case B: Qty growing only (rets NOT growing)
qty_only_growing_mask = growing_mask & (df_action['last_hour_qty_status'] == 'growing') & (df_action['last_hour_rets_status'] != 'growing')

def get_qty_growing_cart_rule(row):
    """Calculate new cart rule for qty growing only."""
    # Calculate qty per retailer
    uth_retailers = row.get('uth_retailers', 0)
    if uth_retailers == 0:
        qty_per_retailer = np.inf
    else:
        qty_per_retailer = row['uth_qty'] / uth_retailers
    
    # Get the three options
    option1 = row.get('normal_refill_3std', np.inf)
    option2 = qty_per_retailer * 0.85
    option3 = row.get('current_cart_rule', np.inf) * 0.85
    
    # Return minimum of the three (excluding inf/nan)
    options = [opt for opt in [option1, option2, option3] if pd.notna(opt) and opt != np.inf]
    if options:
        return min(options)
    return np.nan

df_action.loc[qty_only_growing_mask, 'new_cart_rule'] = df_action[qty_only_growing_mask].apply(get_qty_growing_cart_rule, axis=1)

# Round and apply min/max constraints to new_cart_rule
df_action['new_cart_rule'] = df_action['new_cart_rule'].round()
df_action['new_cart_rule'] = df_action['new_cart_rule'].clip(lower=2, upper=150)

df_action.loc[qty_only_growing_mask, 'cart_rule_action'] = 'qty_growing'

print(f"  Qty growing actions (cart rule change): {qty_only_growing_mask.sum()} SKUs")
print(f"    (Cart rule: rounded, min=2, max=150)")

# =============================================================================
# SUMMARY
# =============================================================================
print(f"\n{'='*60}")
print("ACTION SUMMARY")
print(f"{'='*60}")
print(f"\nPrice Actions:")
print(df_action['price_action'].value_counts().to_string())
print(f"\nCart Rule Actions:")
print(df_action['cart_rule_action'].value_counts().to_string())

# Show SKUs with new prices
price_changes = df_action[df_action['new_price'].notna()]
print(f"\nSKUs with new price: {len(price_changes)}")
if len(price_changes) > 0:
    print(f"  Avg price change: {((price_changes['new_price'] - price_changes['current_price']) / price_changes['current_price']).mean()*100:.1f}%")

# Show SKUs with new cart rules
cart_changes = df_action[df_action['new_cart_rule'].notna()]
print(f"\nSKUs with new cart rule: {len(cart_changes)}")
if len(cart_changes) > 0:
    print(f"  Avg cart rule change: {((cart_changes['new_cart_rule'] - cart_changes['current_cart_rule']) / cart_changes['current_cart_rule']).mean()*100:.1f}%")

# Display sample
print(f"\n{'='*60}")
print("SAMPLE OUTPUT")
print(f"{'='*60}")
output_cols = ['sku', 'current_price', 'new_price', 'current_cart_rule', 'new_cart_rule', 
               'price_action', 'cart_rule_action', 'action_reason']
output_cols = [c for c in output_cols if c in df_action.columns]
display(df_action[df_action['new_price'].notna() | df_action['new_cart_rule'].notna()][output_cols].head(15))


Calculating actions for filtered SKUs...
  WAC increase actions: 253 SKUs
  Rets growing actions (price increase): 905 SKUs
  Qty growing actions (cart rule change): 33 SKUs

ACTION SUMMARY

Price Actions:
price_action
rets_growing    905
wac_increase    253
none             33

Cart Rule Actions:
cart_rule_action
none           1158
qty_growing      33

SKUs with new price: 1149
  Avg price change: 0.8%

SKUs with new cart rule: 33
  Avg cart rule change: -94.2%

SAMPLE OUTPUT


,sku,current_price,new_price,current_cart_rule,new_cart_rule,price_action,cart_rule_action,action_reason
19,مكرونة بساطة فرن - 350 جم,166.75,167.907194,13,NaN,rets_growing,none,growing_performance
23,رويال أعشاب نعناع - 20 فتلة,18.25,18.455194,72,NaN,rets_growing,none,growing_performance
50,أد - مى باربيكيو 430 جم,584.50,477.998034,13,NaN,wac_increase,none,wac_increase
52,اوكسى يدوى نسيم الشرق - 250 جم,138.25,138.808755,25,NaN,wac_increase,none,wac_increase
69,شوكولاتة غامقة دريم خام للطبخ - 200 جم,50.25,53.000000,24,NaN,wac_increase,none,wac_increase
75,صابون لوكس سحر الاوركيد - 165 جم,152.25,154.055419,8,NaN,rets_growing,none,growing_performance
83,-أريال يدوى لافندر 1 كجم,375.75,448.750000,25,NaN,rets_growing,none,growing_performance
124,اد مى مايونيز بالثوم سكويز - 360 جم,601.25,575.357428,13,NaN,wac_increase,none,wac_increase
125,اد مى مايونيز بالثوم سكويز - 360 جم,601.25,575.357428,13,NaN,wac_increase,none,wac_increase
156,الضحى خلطة توابل الحواوشى - 30 جم,134.00,135.213137,5,NaN,rets_growing,none,growing_performance


In [18]:
df_action.to_excel('check.xlsx')

In [16]:
# =============================================================================
# SAMPLE OUTPUT - Current Status
# =============================================================================
# Show sample of data with all calculated fields

sample_cols = [
    'warehouse_id', 'product_id', 'sku','wac1','wac_p','new_wac',
    # P80/P70 benchmarks and std
    'p80_daily_240d', 'std_daily_240d', 'p70_daily_retailers_240d', 'std_daily_retailers_240d',
    # Current cart rule
    'current_cart_rule',
    # UTH performance (with std thresholds)
    'uth_qty', 'uth_qty_target', 'uth_qty_std', 'uth_qty_status',
    'uth_retailers', 'uth_rets_target', 'uth_rets_std', 'uth_rets_status',
    # Last hour performance (with std thresholds)
    'last_hour_qty', 'last_hour_qty_target', 'last_hour_qty_std', 'last_hour_qty_status',
    'last_hour_retailers', 'last_hour_rets_target', 'last_hour_rets_std', 'last_hour_rets_status'
]

# Filter to columns that exist
sample_cols = [c for c in sample_cols if c in df.columns]

print(f"\n{'='*60}")
print("SAMPLE DATA (First 10 rows with UTH > 0)")
print(f"{'='*60}")
sample = df[df['uth_qty'] > 0][sample_cols].head(10)
display(sample)



SAMPLE DATA (First 10 rows with UTH > 0)


,warehouse_id,product_id,sku,wac1,wac_p,new_wac,p80_daily_240d,std_daily_240d,p70_daily_retailers_240d,std_daily_retailers_240d,...,uth_rets_std,uth_rets_status,last_hour_qty,last_hour_qty_target,last_hour_qty_std,last_hour_qty_status,last_hour_retailers,last_hour_rets_target,last_hour_rets_std,last_hour_rets_status
0,339,3934,مسحوق غسيل فل ازرق يدوى - 2.5 كجم,49.402499,45.307242,45.309865,7.0,5.312176,1.0,1.110960,...,0.656389,on_track,0.0,0.532131,0.403825,dropping,0.0,0.072548,0.080598,dropping
1,170,3934,مسحوق غسيل فل ازرق يدوى - 2.5 كجم,49.402499,45.307242,45.309865,4.0,3.970764,1.0,0.816997,...,0.485959,on_track,0.0,0.268193,0.266233,dropping,0.0,0.069006,0.056378,dropping
5,632,2190,أريال باور جل اتوماتيك لافندر (كرتونة 4 عبوة) ...,762.516652,651.323623,651.369034,0.0,0.514917,0.0,0.428785,...,0.259069,growing,0.0,0.000000,0.037507,dropping,0.0,0.000000,0.030210,dropping
6,632,4232,حفاضات جود كير مقاس 6 - 40 حفاضة,199.899077,174.484718,174.484701,12.0,8.305985,3.0,1.568948,...,0.926115,on_track,0.0,0.883288,0.611381,dropping,0.0,0.229447,0.119997,dropping
7,797,2729,شوكولاتة كادبورى سادة - 30 جم,339.701366,312.525257,312.526820,1.0,0.704581,1.0,0.704581,...,0.447086,on_track,0.0,0.058220,0.041021,dropping,0.0,0.057580,0.040570,dropping
9,8,6872,حفاضات مولفيكس حديث الولادة مقاس 1 - 60 حفاضة,277.048092,250.982027,250.988826,12.0,9.013025,3.0,2.378425,...,1.318374,on_track,0.0,0.868870,0.652596,dropping,0.0,0.194572,0.154258,dropping
10,501,9760,مناديل زينة تريو عرض خاص - 500 منديل,432.352567,388.510765,388.511045,4.0,2.004124,2.0,1.303529,...,0.779781,on_track,0.0,0.292533,0.146568,dropping,0.0,0.148298,0.096656,dropping
13,1,2729,شوكولاتة كادبورى سادة - 30 جم,339.701366,312.525257,312.526820,9.4,3.667797,7.0,2.831469,...,1.818937,on_track,0.0,0.551701,0.215269,dropping,0.0,0.448120,0.181263,dropping
16,962,5453,مولتو ماجنم شوكولاتة بالبندق - 15 جنية,280.990000,280.990000,280.990000,18.6,4.949747,11.4,1.414214,...,0.883214,on_track,0.0,1.100013,0.292730,dropping,0.0,0.692918,0.085959,dropping
17,632,7704,مكرونة بساطة شعرية - 350 جم,162.335143,153.954842,153.902763,2.0,1.438776,1.0,0.939455,...,0.572567,on_track,0.0,0.127562,0.091767,dropping,0.0,0.057058,0.053603,dropping


In [17]:
# =============================================================================
# ACTION ENGINE (TO BE DEFINED)
# =============================================================================
# This section will contain the action logic based on:
# - uth_qty_status, uth_rets_status
# - last_hour_qty_status, last_hour_rets_status
#
# Placeholder for now - actions will be defined by user

print(f"\n{'='*60}")
print("MODULE 4 - DATA PREPARATION COMPLETE")
print(f"{'='*60}")
print(f"\nReady for action definition. Available statuses:")
print(f"  - uth_qty_status: {df['uth_qty_status'].unique().tolist()}")
print(f"  - uth_rets_status: {df['uth_rets_status'].unique().tolist()}")
print(f"  - last_hour_qty_status: {df['last_hour_qty_status'].unique().tolist()}")
print(f"  - last_hour_rets_status: {df['last_hour_rets_status'].unique().tolist()}")
print(f"\nTotal records: {len(df)}")



MODULE 4 - DATA PREPARATION COMPLETE

Ready for action definition. Available statuses:
  - uth_qty_status: ['on_track', 'dropping', 'growing']
  - uth_rets_status: ['on_track', 'dropping', 'growing']
  - last_hour_qty_status: ['dropping', 'on_track', 'growing']
  - last_hour_rets_status: ['dropping', 'growing', 'on_track']

Total records: 28437
